In [56]:
! ls -alh/var/genie- in -the-box/src/conf/long-term-memory/solutions

total 52K
drwxr-xr-x 2 1000 1000 4.0K Aug 18 15:48 .
drwxr-xr-x 3 1000 1000 4.0K Aug 17 22:53 ..
-rw-r--r-- 1 1000 1000  41K Aug 18 15:39 what-day-is-today-0.json


In [57]:
# ! pip install python-dotenv
! pip install langchain
! pip install langchain[ docarray ]
! pip install tiktoken
! pip install pinecone-client
! pip install jq
! pip install chromadb
! pip install openai[ embeddings ]


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [113]:
import sys
import os

import pandas as pd
import json

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )

import util as du
import util_stopwatch as sw
import util_langchain as ulc
import solution_snapshot_mgr as ssm
import solution_snapshot as ss

path = "/var/genie-in-the-box/src"
du.add_to_path( path )
sys.path

Path [/var/genie-in-the-box/src] already in sys.path


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug',
 '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev',
 '/var',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/var/genie-in-the-box/src/lib',
 '/var/genie-in-the-box/src/lib',
 '/var/genie-in-the-box/src']

In [114]:
timer = sw.Stopwatch()
timer.print( "Foo completed!", use_millis=True )

Foo completed! in 0 ms


In [115]:
# reload source code using inline magic for Jupiter notebook
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [372]:
%autoreload
path_to_snapshots = os.path.join( du.get_project_root(), "src/conf/long-term-memory/solutions/" )
ss_manager = ssm.SolutionSnapshotManager( path_to_snapshots )
print( ss_manager )

Loading question embeddings dictionary ...... Done!
[2] snapshots by question loaded from [/var/genie-in-the-box/src/conf/long-term-memory/solutions/]


In [190]:
ss_manager.print_snapshots()


------------------------------------------------------------------------------------------------------------------------
- Total snapshots: [1]
------------------------------------------------------------------------------------------------------------------------

Question: [what day is today]


In [119]:
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [120]:
# Load CSV data From file into the Pandas DataFrame.
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   start_date                  200 non-null    object
 1   end_date                    200 non-null    object
 2   start_time                  200 non-null    object
 3   end_time                    200 non-null    object
 4   event_type                  200 non-null    object
 5   recurrent                   200 non-null    bool  
 6   recurrence_interval         47 non-null     object
 7   priority_level              200 non-null    object
 8   name                        200 non-null    object
 9   relationship                200 non-null    object
 10  description_who_what_where  200 non-null    object
dtypes: bool(1), object(10)
memory usage: 15.9+ KB


In [316]:
def get_event_count( df, event ):
    count = df[ df.event_type == event ].shape[ 0 ]
    print( "Total {} {} events".format( count, event ) )

get_event_count( df, "Birthday" )

Total 16 Birthday events


In [217]:
event_prompt_template_string = """
df contains two important fields:
1) event_type: tells us what kind of event is taking place, such as birthday, conference, meeting, etc.
2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.
Query df and answer in plain English unless requested not to.
NEVER display raw dataframe content, unless requested not to.
All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.
When reporting to me, you will always sort events in ascending order, 1 per line, unless requested not to.
When summarizing your results you will include the event type, start time, date (e.g.: today, tomorrow, Wednesday the 10th, etc.), and what the event is about, using conversational English, unless requested not to.
DO NOT prepend numbers to a list of events, unless requested not to.

{solution_snapshot_text}

Format your response as a json object containing the following fields
1) question: the question that was asked
2) answer: the answer to the question
3) solution_summary: a plain english summary of how you arrived at your answer to the question. This summary should be no more than 3 sentences long, and should not include the answer to the question.
4) code: the code used to answer the question, formatted as a list of strings, one string per line.
5) programming_language: the programming language used to answer the question
6) language_version: the programming language version used to answer the question
7) coding_approach: whether you executed code 'verbatim' from a previous solution, 'modified' code from a previous solution, or created 'new code'
"""

exact_solution_history_string = """You have successfully answered this same question before.

This is your summary of the solution:

{solution_summary}

This is the code you used to answer the question:

{solution_code}

Please execute the code verbatim.
Once you have done that, debug it if it fails.
"""

similar_solution_history_string = """You have successfully answered a question similar to this before.

This is your summary of the solution:

{solution_summary}

This is the code you used to answer the question:

{solution_code}

Please examine the summary and the code and then modify the code to solve the current question.
Once you have done that, execute the code. If it fails, debug it.
"""

event_prompt_template = ChatPromptTemplate.from_template( event_prompt_template_string )

In [218]:
def get_solution_snapshot( question ):

    similar_snapshots = ss_manager.get_snapshots_by_question( question )
    
    if len( similar_snapshots ) > 0:
        
        solution_summary = similar_snapshots[ 0 ][ 1 ].solution_summary
        solution_code = "\n".join( similar_snapshots[ 0 ][ 1 ].code )
        
        # if the confidence is 100% use the exact solution history
        if similar_snapshots[ 0 ][ 0 ] == 100.0:
            solution_history_text = exact_solution_history_string.format( solution_summary=solution_summary, solution_code=solution_code )
        else:
            solution_history_text = similar_solution_history_string.format( solution_summary=solution_summary, solution_code=solution_code)
    else:
        solution_history_text = ""
    
    return solution_history_text


# question = "What's today's date?"
question = "What day is today?"
# question = "What day was yesterday?"
# question = "Why is the sky blue?"

solution_snapshot_text = get_solution_snapshot( question )
solution_snapshot_text

get_snapshots_by_question( 'what day is today' )...
Exact match: Snapshot with question [what day is today] exists!
Found [1] similar snapshots
score [100.0] for [what day is today] == [what day is today]


"You have successfully answered this same question before.\n\nThis is your summary of the solution:\n\nI used Python's datetime library to get the current date.\n\nThis is the code you used to answer the question:\n\nimport datetime\ntoday = datetime.date.today()\ntoday\n\nPlease execute the code verbatim.\nOnce you have done that, debug it if it fails.\n"

In [219]:
def get_event_prompt( question ):
    
    question = ss.SolutionSnapshot.clean_question( question )
    solution_snapshot_text = get_solution_snapshot( question )
    
    event_prompt = event_prompt_template.format_messages( question=question, solution_snapshot_text=solution_snapshot_text )
    
    return event_prompt

event_prompt = get_event_prompt( "What day is today?" )
# event_prompt = get_event_prompt( "Why is the sky blue?" )

print( event_prompt, end="\n\n" )

get_snapshots_by_question( 'what day is today' )...
Exact match: Snapshot with question [what day is today] exists!
Found [1] similar snapshots
score [100.0] for [what day is today] == [what day is today]
[HumanMessage(content="\nYou are an event planner with proven capacity for querying pandas dataframes.\nThe dataframe that I am providing you for inspection contains two important fields:\n1) event_type: tells us what kind of event is taking place, such as birthday, conference, meeting, etc.\n2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.\nI expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.\nNEVER display raw dataframe content, unless specifically requested to do otherwise.\nAll dates and temporal references like 'today', 'yester

In [252]:
vars( event_prompt )

TypeError: vars() argument must have __dict__ attribute

In [243]:
llm = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0, verbose=True )
df_agent = create_pandas_dataframe_agent( llm, df, verbose=True, return_intermediate_steps=True )
df_agent.agent.llm_chain.verbose = True

Before: ['import datetime', 'today = datetime.date.today()', 'today']
 After: ['import datetime', 'today = datetime.date.today()', 'print( today )']


In [253]:
event_prompt = get_event_prompt( "how many records in this dataframe?" )

print( event_prompt, end="\n\n" )

get_snapshots_by_question( 'how many records in this dataframe' )...
Exact match: Snapshot with question [how many records in this dataframe] exists!
Found [1] similar snapshots
score [100.0] for [how many records in this dataframe] == [how many records in this dataframe]
[HumanMessage(content="\nYou are an event planner with proven capacity for querying pandas dataframes.\nThe dataframe that I am providing you for inspection contains two important fields:\n1) event_type: tells us what kind of event is taking place, such as birthday, conference, meeting, etc.\n2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.\nI expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.\nNEVER display raw dataframe content, unless specifically requested to do

In [245]:
# ss_manager.delete_snapshot( "how many records in this dataframe?" )
vars( df_agent.agent.llm_chain )

{'memory': None,
 'callbacks': None,
 'callback_manager': None,
 'verbose': True,
 'tags': None,
 'metadata': None,
 'prompt': PromptTemplate(input_variables=['agent_scratchpad', 'input'], output_parser=None, partial_variables={'df_head': '|    | start_date   | end_date   | start_time   | end_time   | event_type   | recurrent   |   recurrence_interval | priority_level   | name       | relationship   | description_who_what_where                      |\n|---:|:-------------|:-----------|:-------------|:-----------|:-------------|:------------|----------------------:|:-----------------|:-----------|:---------------|:------------------------------------------------|\n|  0 | 2023-07-01   | 2023-07-04 | 00:00        | 23:59      | Concert      | False       |                   nan | none             | Jenny      | coworker       | Concert of Jenny at the city center             |\n|  1 | 2023-07-01   | 2023-07-01 | 05:25        | 17:22      | TODO         | False       |                   na

In [249]:
events_preemble = """
df contains two important fields:
1) event_type: tells us what kind of event is taking place, such as birthday, conference, meeting, etc.
2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.
Answer in plain English.
NEVER display raw dataframe content.
All dates and temporal references like 'today', 'yesterday', 'next week' should be calculated by using Python's datetime library.
You will always sort df's events in ascending order, 1 per line.
When summarizing your results include the event type, start time, date (e.g.: today, tomorrow, Wednesday the 10th, etc.), and what the event is about, using conversational Englishd.
DO NOT prepend numbers to a list of events.


"""

format_instructions = """{solution_snapshot_text}

Format your response as a json object containing the following fields
1) question: the question that was asked
2) answer: the answer to the question
3) solution_summary: a plain english summary of how you arrived at your answer to the question. This summary should be no more than 3 sentences long, and should not include the answer to the question.
4) code: the code used to answer the question, formatted as a list of strings, one string per line.
5) programming_language: the programming language used to answer the question
6) language_version: the programming language version used to answer the question
7) coding_approach: whether you executed code 'verbatim' from a previous solution, 'modified' code from a previous solution, or created 'new code'
"""

exact_solution_history_string = """You have successfully answered this same question before.

This was your summary of the solution:

{solution_summary}

This is the code you used to answer the question:

{solution_code}

Please execute the code verbatim.
Once you have done that, debug it if it fails.
"""

similar_solution_history_string = """You have successfully answered a question similar to this before.

This is your summary of the solution:

{solution_summary}

This is the code you used to answer the question:

{solution_code}

Please examine the summary and the code and then modify the code to solve the current question.
Once you have done that, execute the code. If it fails, debug it.
"""

event_prompt_template = ChatPromptTemplate.from_template( event_prompt_template_string )

In [251]:
template='\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

my_template = template.replace( "{df_head}", "{df_head}" + "\n\n" + events_preemble  )

lines = my_template.split( "\n" )
for line in lines:
    print( line )


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}


df contains two important fields:
1) event_type: tells us what kind of event is taking place, such as birthday, conferen

In [241]:
vars( df_agent.agent.llm_chain )[ "prompt" ].template = "foo?"

In [244]:
vars( df_agent.agent.llm_chain )[ "prompt" ].template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [223]:
from langchain.callbacks import StdOutCallbackHandler

# Create function to take a question and call the data frame agent.
def query_dataframe( question ):
    
    timer = sw.Stopwatch()
    
    # event_prompt = event_prompt_template.format_messages( question=question )
    event_prompt = get_event_prompt( question )
    # response = df_agent.run( event_prompt )
    callbacks = [ StdOutCallbackHandler() ]
    response = df_agent( { "input": event_prompt }, callbacks=callbacks )
    
    timer.print( "Query completed!", use_millis=True )
    
    return response


response = query_dataframe( "how many records in this dataframe?" )
# response = query_dataframe( "What day is today?" )
# response = query_dataframe( "What day was yesterday?" )
# response = query_dataframe( "What day is tomorrow?" )

response

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).
Error in StdOutCallbackHandler.on_retry callback: 'StdOutCallbackHandler' object has no attribute 'on_retry'


get_snapshots_by_question( 'how many records in this dataframe' )...
Exact match: Snapshot with question [how many records in this dataframe] exists!
Found [1] similar snapshots
score [100.0] for [how many records in this dataframe] == [how many records in this dataframe]


> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observatio

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Error in StdOutCallbackHandler.on_retry callback: 'StdOutCallbackHandler' object has no attribute 'on_retry'


KeyboardInterrupt: 

# Stash completed chains output
get_snapshots_by_question( 'how many records in this dataframe' )...
Embedding for question [how many records in this dataframe] already exists!
Could not find any snapshots similar to [how many records in this dataframe]


[1m> Entering new AgentExecutor chain...[0m
[32;1m[1;3mThought: The question is asking for the total number of records in the dataframe. In pandas, we can use the `shape` attribute to get the number of rows and columns in the dataframe. The number of rows corresponds to the number of records.
Action: python_repl_ast
Action Input: df.shape[0][0m
Observation: [36;1m[1;3m200[0m
Thought:[32;1m[1;3mThe dataframe has 200 records.
Final Answer: {"question": "how many records in this dataframe", "answer": 200, "solution_summary": "I used the shape attribute of the pandas dataframe to get the number of records.", "code": ["df.shape[0]"], "programming_language": "Python", "language_version": "3.7", "coding_approach": "new code"}[0m

[1m> Finished chain.[0m
Query completed! in 16,841 ms

'{"question": "how many records in this dataframe", "answer": 200, "solution_summary": "I used the shape attribute of the pandas dataframe to get the number of records.", "code": ["df.shape[0]"], "programming_language": "Python", "language_version": "3.7", "coding_approach": "new code"}'

In [198]:
response = query_dataframe( "how many records in this dataframe?" )

get_snapshots_by_question( 'how many records in this dataframe' )...
Exact match: Snapshot with question [how many records in this dataframe] exists!
Found [1] similar snapshots
score [100.0] for [how many records in this dataframe] == [how many records in this dataframe]


> Entering new AgentExecutor chain...
Thought: The question is asking for the number of records in the dataframe. This can be obtained by using the shape attribute of the dataframe, which returns a tuple where the first element is the number of rows (records) and the second element is the number of columns. We only need the first element of this tuple.
Action: [python_repl_ast]
Action Input: df.shape[0]
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:I made a mistake in choosing the tool. I should use python_repl_ast to execute the Python command.
Action: [python_repl_ast]
Action Input: df.shape[0]
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_a

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-sfQFuSQezrFbiAFD4jrqCebt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


I made a mistake in choosing the tool. I should use python_repl_ast to execute the Python command.
Action: [python_repl_ast]
Action Input: df.shape[0]
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-sfQFuSQezrFbiAFD4jrqCebt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-sfQFuSQezrFbiAFD4jrqCebt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


I made a mistake in choosing the tool. I should use python_repl_ast to execute the Python command.
Action: [python_repl_ast]
Action Input: df.shape[0]
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:

...

I made a mistake in choosing the tool. I should use python_repl_ast to execute the Python command.
Action: [python_repl_ast]
Action Input: df.shape[0]
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-sfQFuSQezrFbiAFD4jrqCebt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


I made a mistake in choosing the tool. I should use python_repl_ast to execute the Python command.
Action: [python_repl_ast]
Action Input: df.shape[0]
Observation: [python_repl_ast] is not a valid tool, try one of [python_repl_ast].
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-sfQFuSQezrFbiAFD4jrqCebt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-sfQFuSQezrFbiAFD4jrqCebt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


KeyboardInterrupt: 

In [199]:
# parse the json string into a Python dictionary
response_dict = json.loads( response )
print( response_dict )
raw_code = response_dict[ "code" ]
# print code line by line
for line in raw_code:
    print( line )

{'question': 'how many records in this dataframe', 'answer': 200, 'solution_summary': 'I used the shape attribute of the pandas dataframe to get the number of records.', 'code': ['df.shape[0]'], 'programming_language': 'Python', 'language_version': '3.7', 'coding_approach': 'verbatim'}
df.shape[0]


In [195]:
%autoreload

def add_solution_snapshot( response ):
    
    # parse the json string into a Python dictionary
    response = json.loads( response )
    
    solution_snapshot = ss.SolutionSnapshot(  
                             response[ "question" ], 
            solution_summary=response[ "solution_summary" ], 
                        code=response[ "code" ], 
        programming_language=response[ "programming_language" ], 
            language_version=response[ "language_version" ], 
          question_embedding=ss_manager.get_question_embedding( response[ "question" ] ) 
    )
    
    ss_manager.add_snapshot( solution_snapshot )

add_solution_snapshot( response )

NO solution_file value provided (Must be a new object). Generating a unique file name...
File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/how-many-records-in-this-dataframe-0.json


In [365]:
%autoreload
ss_manager.print_snapshots()


------------------------------------------------------------------------------------------------------------------------
- Total snapshots: [2]
------------------------------------------------------------------------------------------------------------------------

Question: [what day is today]
Question: [how many records in this dataframe]


In [186]:
ss_manager.delete_snapshot( "What day is today?" )

Deleting snapshot with question [what day is today]...
Snapshot with question [what day is today] deleted!


True

In [332]:
def force_print_cmd( code ):
    
    if "print(" not in code[ -1 ]:
        code[ -1 ] = "print( {} )".format( code[ -1 ] )
    return code

code = [
"import datetime",
"today = datetime.date.today()",
"today"
]
print( "Before:", code )
code = force_print_cmd( code )
print( " After:", code )
    

Before: ['import datetime', 'today = datetime.date.today()', 'today']
 After: ['import datetime', 'today = datetime.date.today()', 'print( today )']


In [361]:
from subprocess import PIPE, run

def force_print_cmd( code ):
    
    if "print(" not in code[ -1 ]:
        code[ -1 ] = "print( {} )".format( code[ -1 ] )
    return code

def assemble_and_run_solution( solution_code, path, debug=True ):
    
    code_preamble = [
        "import pandas as pd",
        "df = pd.read_csv( \"{path}\" )".format( path=path )
    ]
    
    if debug: print( "last command, before [{}]:".format( solution_code[ -1 ] ) )
    solution_code = force_print_cmd( solution_code )
    if debug: print( "last command,  after [{}]:".format( solution_code[ -1 ] ) )
    
    code = code_preamble + solution_code + [ "" ]
    
    code_path = du.get_project_root() + "/io/code.py"
    du.write_lines_to_file( code_path, code )
    
    if debug: print( "Executing {}... ".format( code_path ), end="" )
    results = run( [ "python3", code_path ], stdout=PIPE, stderr=PIPE, universal_newlines=True )
    
    if results.returncode != 0:
        print()
        response = "\nERROR:\n{}".format( results.stderr.strip() )
    else:
        if debug: print( "Done!" )
        response = results.stdout.strip()
        
    results_dict = {
        "return_code": results.returncode,
           "response": response
    }
    return results_dict
        
solution_code = [
    "num_records = df.shape[0]",
    "num_records"
]        
results = assemble_and_run_solution( solution_code, du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

if results[ "return_code" ] != 0:
    print( results[ "response" ] )
else:
    response = results[ "response" ]    
    for line in response.split( "\n" ): print( line )

last command, before [num_records]:
last command,  after [print( num_records )]:
Executing /var/genie-in-the-box/io/code.py... Done!
200


In [375]:
%autoreload
ulc.test_assemble_and_run_solution()

last command, before [num_records]:
last command,  after [print( num_records )]:
Executing /var/genie-in-the-box/io/code.py... Done!
200


In [381]:
solution_snapshot = ss_manager.get_snapshots_by_question( "why is the sky blue?" )
solution_snapshot

get_snapshots_by_question( 'why is the sky blue' )...
Embedding for question [why is the sky blue] already exists!
Could not find any snapshots similar to [why is the sky blue]


[]

In [382]:
solution_snapshot = ss_manager.get_snapshots_by_question( "how many records in this dataframe?" )

solution      = solution_snapshot[ 0 ][ 1 ]
solution      = json.loads( solution.to_json() )
solution_code = solution[ "code" ]


get_snapshots_by_question( 'how many records in this dataframe' )...
Exact match: Snapshot with question [how many records in this dataframe] exists!
Found [1] similar snapshots
score [100.0] for [how many records in this dataframe] == [how many records in this dataframe]


In [409]:
%autoreload

def run_solution( solution_snapshot, df_path=None, debug=False ):
    
    question      = solution_snapshot[ 0 ][ 1 ].question
    solution_code = solution_snapshot[ 0 ][ 1 ].code
    
    if df_path is not None:
        df_path = du.get_project_root() + df_path
        
    results = ulc.assemble_and_run_solution( solution_code, df_path, debug=debug )
    results[ "question" ] = question
        
    if debug:
        
        if results[ "return_code" ] != 0:
            print( results[ "response" ] )
        else:
            response = results[ "response" ]
            print()
            print( f"Q: {question}" )
            for line in response.split( "\n" ): 
                print( f"A: {line}" )
            
    
    return results

question = "How many records in this dataframe?"
# question = "What day is today?"
debug   = False
df_path = "/src/conf/long-term-memory/events.csv"
# df_path = None

solution_snapshot = ss_manager.get_snapshots_by_question( question, debug=debug )
results = run_solution( solution_snapshot, df_path=df_path, debug=debug )
results

{'return_code': 0,
 'response': '200',
 'question': 'how many records in this dataframe'}

In [338]:
solution_snapshot =! pwd
! ls -alh /var/genie-in-the-box/io

/var
total 28K
drwxr-xr-x  2 1000 1000 4.0K Aug 23 15:44 .
drwxr-xr-x 10 1000 1000 4.0K Aug 21 20:59 ..
-rw-r--r--  1 1000 1000  11K Aug  1 19:44 chat_history.txt
-rwxr--r--  1 root root  148 Aug 23 17:52 code.py
-rw-r--r--  1 root root  285 Aug 23 17:13 last_response.json
